# Notebook Setup

In [1]:
import pandas as pd
import numpy as np

#import matplotlib.pyplot as plt
#import seaborn as sns
#from rake_nltk import Rake

#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [5]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

In [4]:
#!pip freeze|grep tensorflow
#!pip uninstall numpy
!pip install ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/b8/d4/ce436660098b2f456e2b8fdf76d4f33cbc3766c874c4aa2f772c7a5e943f/ipywidgets-8.1.0-py3-none-any.whl.metadata
  Using cached ipywidgets-8.1.0-py3-none-any.whl.metadata (2.4 kB)
  Obtaining dependency information for widgetsnbextension~=4.0.7 from https://files.pythonhosted.org/packages/8e/d4/d31b12ac0b87e8cc9fdb6ea1eb6596de405eaaa2f25606aaa755d0eebbc0/widgetsnbextension-4.0.8-py3-none-any.whl.metadata
  Using cached widgetsnbextension-4.0.8-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for jupyterlab-widgets~=3.0.7 from https://files.pythonhosted.org/packages/74/5e/2475ac62faf2e342b2bf20b8d8e375f49400ecb38f52e4e0a7557eb1cedb/jupyterlab_widgets-3.0.8-py3-none-any.whl.metadata
  Using cached jupyterlab_widgets-3.0.8-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.0-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.8-py3-none-any.whl (214 kB)
Us

# Import Data

In [6]:
df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/data.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['text_len'] = df['abstract_text'].apply(lambda x: len(x))

In [ ]:
sns.histplot(df['text_len'])

# BERTopic Model

## setup model

In [7]:
representation_model = KeyBERTInspired()
topic_model = BERTopic()#(representation_model=representation_model)

In [8]:
size = 200

df_small = df.iloc[0:size]

df_train = df_small['abstract_text']
df_train

0      We conducted this study to assess the clinical...
1      To determine whether prophylactic treatment wi...
2      After the discovery of type C hepatitis virus ...
3      Since it is not clear whether testosterone or ...
4      The aim was to study the pharmacokinetic param...
                             ...                        
195    To investigate the sensitivity , specificity a...
196    To study the influence of the position of the ...
197    To compare the clinical efficacy , patient sat...
198    To assess whether intervention by a health vis...
199    To evaluate the effect of short term treatment...
Name: abstract_text, Length: 200, dtype: object

In [9]:
%%time

topics, probs = topic_model.fit_transform(df_train)

CPU times: user 1min 2s, sys: 7.23 s, total: 1min 9s
Wall time: 26.6 s


## Explore model

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info

In [ ]:
topic_model.get_topic(1)

In [ ]:
topic_model.get_topic_freq()


In [ ]:
topic_model.generate_topic_labels()

In [ ]:
topic_model.find_topics("aricle about lung cancer patient care")

In [ ]:
topic_model.hierarchical_topics(df_small['abstract_text'])

In [ ]:
#from scipy.cluster import hierarchy as sch

linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(df_train, linkage_function=linkage_function)
#%matplotlib inline
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
#plt.show();

In [ ]:
#tree = topic_model.get_topic_tree(hierarchical_topics)
#print(tree)
#topic_model.visualize_hierarchy()

In [ ]:
topic_model.get_document_info(df_train)

## tag new articles

In [ ]:
tag_size = 500

df_new = df['abstract_text'].iloc[size:size+tag_size]

topic_model.transform(df['abstract_text'].iloc[size+1])

## Extract topic from query

In [ ]:
#df_with_topics = pd.concat([df_small,pd.DataFrame({'topic_id':topics,'probability':probs})],axis=1)
df_with_topics = pd.concat([df_small['abstract_id'],topic_model.get_document_info(df_train)],axis=1)
df_with_topics

In [ ]:
def url_destination(id):
    #ULR example: 'https://pubmed.ncbi.nlm.nih.gov/16364933/'
    url_template = 'https://pubmed.ncbi.nlm.nih.gov/'
    return f"{url_template}{id}"
    
def find_article(query,model):
    # Find topics from query
    f_topics, f_prob = model.find_topics(query)
    topic_info = topic_model.get_topic_info()

    # extarct the optiosn from the DB
    for t in range(len(f_topics)):
        topic_id = f_topics[t]
        topic_prob = round(f_prob[t]*100,2)
        topic_name = topic_info['Name'][topic_info['Topic'] == topic_id].values[0]
        article_count = df_with_topics['abstract_id'][df_with_topics['Topic'] == t].count()
        print(f"Recommended Topics: {topic_name} with a probability of {topic_prob}% & we've found {article_count} articles\n")

    # Ask user for topic selection
    selected_id = input('select a topic ID to show the articles:')

    # Generate the article destination URL + display the options
    article_list = df_with_topics[df_with_topics['Topic'] == int(selected_id)]#.count()
    article_list['article_link'] = article_list['abstract_id'].apply(url_destination)
    display(article_list[['Document','article_link']])

In [ ]:
query = 'article reviewing stomach cancer care treatments with clinical data'

find_article(query,topic_model)

In [15]:
#%matplotlib inline

#topic_model.visualize_topics()
#fig.show().show()
tree = topic_model.get_topic_tree(topic_model.hierarchical_topics(df_train))
print(tree)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 124.95it/s]

.
├─■──of_the_in_and_were ── Topic: 4
└─the_of_and_in_to
     ├─the_of_and_in_to
     │    ├─■──of_the_and_to_in ── Topic: 2
     │    └─the_of_in_and_to
     │         ├─■──the_in_and_of_to ── Topic: 0
     │         └─■──of_the_and_in_patients ── Topic: 1
     └─■──the_of_and_patients_survival ── Topic: 3



In [11]:
import plotly.io as pio
pio.renderers.default='notebook'